In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [73]:
space = pd.read_json('files/spaceX_Lunch.json')

Observaciones:
- ships/capsules/payloads: están formadas por listas
- crew/cores/failures: lista de diccionarios
- fairings: diccionario
- links: diccionario de diccionario

In [74]:
space.head(2)

,fairings,links,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id
0,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a
1,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight'...",True,False,None,5eb87cdaffd86e000604b32b


In [ ]:
space['ships'].unique() #como es una lista no podemos ver lo que hay

TypeError: unhashable type: 'list'

In [10]:
space['ships'].value_counts().sum()

205

In [11]:
space['ships'].value_counts()

ships
[]                                                                                                                                    85
[5ea6ed2e080df4000697c906, 5ea6ed2f080df4000697c90b, 5ea6ed2f080df4000697c90c, 5ea6ed30080df4000697c913]                               6
[5ea6ed30080df4000697c912]                                                                                                             6
[5ea6ed2d080df4000697c902]                                                                                                             4
[5ea6ed30080df4000697c913]                                                                                                             4
[5ea6ed2e080df4000697c908]                                                                                                             4
[5ea6ed2f080df4000697c90d, 5ea6ed30080df4000697c913]                                                                                   3
[5ea6ed2e080df4000697c906, 5ea6ed2f

In [75]:
space.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fairings               169 non-null    object 
 1   links                  205 non-null    object 
 2   static_fire_date_utc   121 non-null    object 
 3   static_fire_date_unix  121 non-null    float64
 4   net                    205 non-null    bool   
 5   window                 117 non-null    float64
 6   rocket                 205 non-null    object 
 7   success                186 non-null    float64
 8   failures               205 non-null    object 
 9   details                134 non-null    object 
 10  crew                   205 non-null    object 
 11  ships                  205 non-null    object 
 12  capsules               205 non-null    object 
 13  payloads               205 non-null    object 
 14  launchpad              205 non-null    object 
 15  flight

In [22]:
space[space['fairings'].isnull()]


,fairings,links,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id
6,None,{'patch': {'small': 'https://images2.imgbox.co...,2010-12-04T00:00:00.000Z,1.291421e+09,False,0.0,5e9d0d95eda69973a809d1ec,1.0,[],None,[],[5ea6ed2d080df4000697c901],[5e9e2c5bf35918ed873b2664],"[5eb0e4b9b6c3bb0006eeb1e8, 5eb0e4b9b6c3bb0006e...",5e9e4501f509094ba4566f84,7,COTS 1,2010-12-08T15:43:00.000Z,1291822980,2010-12-08T11:43:00-04:00,hour,False,"[{'core': '5e9e289ef35918187c3b2629', 'flight'...",True,False,None,5eb87cdeffd86e000604b330
7,None,{'patch': {'small': 'https://images2.imgbox.co...,2012-04-30T00:00:00.000Z,1.335744e+09,False,0.0,5e9d0d95eda69973a809d1ec,1.0,[],"Launch was scrubbed on first attempt, second l...",[],[5ea6ed2d080df4000697c901],[5e9e2c5bf3591882af3b2665],[5eb0e4bab6c3bb0006eeb1ea],5e9e4501f509094ba4566f84,8,COTS 2,2012-05-22T07:44:00.000Z,1335944640,2012-05-22T03:44:00-04:00,hour,False,"[{'core': '5e9e289ef35918f39c3b262a', 'flight'...",True,False,None,5eb87cdfffd86e000604b331
8,None,{'patch': {'small': 'https://images2.imgbox.co...,2012-09-29T00:00:00.000Z,1.348877e+09,False,0.0,5e9d0d95eda69973a809d1ec,1.0,[],"CRS-1 successful, but the secondary payload wa...",[],[5ea6ed2d080df4000697c902],[5e9e2c5bf3591835983b2666],"[5eb0e4bab6c3bb0006eeb1eb, 5eb0e4bab6c3bb0006e...",5e9e4501f509094ba4566f84,9,CRS-1,2012-10-08T00:35:00.000Z,1349656500,2012-10-08T20:35:00-04:00,hour,False,"[{'core': '5e9e289ff3591821a73b262b', 'flight'...",True,False,None,5eb87ce0ffd86e000604b332
9,None,{'patch': {'small': 'https://images2.imgbox.co...,2013-02-25T18:30:00.000Z,1.361817e+09,False,0.0,5e9d0d95eda69973a809d1ec,1.0,[],Last launch of the original Falcon 9 v1.0 laun...,[],[5ea6ed2d080df4000697c902],[5e9e2c5bf359189ef23b2667],[5eb0e4bbb6c3bb0006eeb1ed],5e9e4501f509094ba4566f84,10,CRS-2,2013-03-01T19:10:00.000Z,1362165000,2013-03-01T15:10:00-04:00,hour,False,"[{'core': '5e9e289ff3591884e03b262c', 'flight'...",True,False,None,5eb87ce1ffd86e000604b333
13,None,{'patch': {'small': 'https://images2.imgbox.co...,2014-03-08T00:00:00.000Z,1.394237e+09,False,0.0,5e9d0d95eda69973a809d1ec,1.0,[],"Following second-stage separation, SpaceX cond...",[],[5ea6ed2d080df4000697c902],[5e9e2c5bf3591859a63b2668],[5eb0e4bbb6c3bb0006eeb1f1],5e9e4501f509094ba4566f84,14,CRS-3,2014-04-18T19:25:00.000Z,1397849100,2014-04-18T15:25:00-04:00,hour,False,"[{'core': '5e9e289ff3591829343b2630', 'flight'...",True,False,None,5eb87ce4ffd86e000604b337
17,None,{'patch': {'small': 'https://images2.imgbox.co...,2014-09-17T00:00:00.000Z,1.410912e+09,False,0.0,5e9d0d95eda69973a809d1ec,1.0,[],None,[],[5ea6ed2d080df4000697c902],[5e9e2c5bf3591880643b2669],[5eb0e4bcb6c3bb0006eeb1f5],5e9e4501f509094ba4566f84,18,CRS-4,2014-09-21T05:52:00.000Z,1411278720,2014-09-21T01:52:00-04:00,hour,False,"[{'core': '5e9e28a0f359184a683b2634', 'flight'...",True,False,None,5eb87ce7ffd86e000604b33b
18,None,{'patch': {'small': 'https://images2.imgbox.co...,2014-12-19T00:00:00.000Z,1.418947e+09,False,0.0,5e9d0d95eda69973a809d1ec,1.0,[],"Following second stage separation, SpaceX perf...",[],"[5ea6ed2e080df4000697c906, 5ea6ed2f080df400069...",[5e9e2c5bf35918165f3b266a],[5eb0e4bdb6c3bb0006eeb1f6],5e9e4501f509094ba4566f84,19,CRS-5,2015-01-10T09:47:00.000Z,1420883220,2015-01-10T05:47:00-04:00,hour,False,"[{'core': '5e9e28a0f359187a3c3b2635', 'flight'...",True,False,None,5eb87ce8ffd86e000604b33c
21,None,{'patch': {'small': 'https://images2.imgbox.co...,2015-04-11T00:00:00.000Z,1.428710e+09,False,0.0,5e9d0d95eda69973a809d1ec,1.0,[],"Following the first-stage boost, SpaceX attemp...",[],"[5ea6ed2e080df4000697c906, 5ea6ed2f080df400069...",[5e9e2c5cf359188bfb3b266b],[5eb0e4bdb6c3bb0006eeb1fa],5e9e4501f509094ba4566f84,22,CRS-6,2015-04-14T20:10:00.000Z,1429042200,2015-04-14T16:10:00-04:00,hour,False,"[{'core': '5e9e28a1f359186d533b2638', 'fligh

In [25]:
space.loc[space['fairings'].isnull().sum(), 'fairings']

{'reused': False, 'recovery_attempt': False, 'recovered': False, 'ships': []}

In [24]:
space.loc[space['fairings'].isnull(), 'fairings']

6      None
7      None
8      None
9      None
13     None
17     None
18     None
21     None
23     None
27     None
31     None
35     None
40     None
44     None
50     None
58     None
63     None
71     None
77     None
81     None
84     None
90     None
93     None
106    None
109    None
122    None
128    None
132    None
134    None
135    None
142    None
155    None
158    None
172    None
186    None
195    None
Name: fairings, dtype: object

In [26]:
space['fairings'].isnull().sum()


36

In [27]:
space.loc[0:206, 'fairings']

0      {'reused': False, 'recovery_attempt': False, '...
1      {'reused': False, 'recovery_attempt': False, '...
2      {'reused': False, 'recovery_attempt': False, '...
3      {'reused': False, 'recovery_attempt': False, '...
4      {'reused': False, 'recovery_attempt': False, '...
5      {'reused': None, 'recovery_attempt': None, 're...
6                                                   None
7                                                   None
8                                                   None
9                                                   None
10     {'reused': False, 'recovery_attempt': False, '...
11     {'reused': False, 'recovery_attempt': False, '...
12     {'reused': False, 'recovery_attempt': False, '...
13                                                  None
14     {'reused': False, 'recovery_attempt': False, '...
15     {'reused': False, 'recovery_attempt': False, '...
16     {'reused': False, 'recovery_attempt': False, '...
17                             

In [29]:
space["fairings"].apply(pd.Series).iloc[:, 0:3].head()

,reused,recovery_attempt,recovered
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False


In [30]:
space[["reused","recovery_attempt","recovered"]] = space["fairings"].apply(pd.Series).iloc[:, 0:3]

In [31]:
space.head(2)

,fairings,links,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,reused,recovery_attempt,recovered
0,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False
1,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight'...",True,False,None,5eb87cdaffd86e000604b32b,False,False,False


In [34]:
space.loc[space['reused'].isnull(), 'reused']

5      None
6       NaN
7       NaN
8       NaN
9       NaN
13      NaN
17      NaN
18      NaN
21      NaN
23      NaN
27      NaN
31      NaN
35      NaN
40      NaN
44      NaN
50      NaN
58      NaN
63      NaN
71      NaN
75     None
77      NaN
81      NaN
84      NaN
87     None
90      NaN
93      NaN
96     None
97     None
98     None
100    None
101    None
105    None
106     NaN
107    None
109     NaN
116    None
117    None
120    None
122     NaN
128     NaN
132     NaN
134     NaN
135     NaN
136    None
137    None
138    None
139    None
140    None
141    None
142     NaN
143    None
144    None
145    None
146    None
147    None
148    None
149    None
150    None
151    None
152    None
153    None
154    None
155     NaN
156    None
157    None
158     NaN
159    None
160    None
161    None
162    None
163    None
164    None
165    None
166    None
167    None
168    None
169    None
170    None
171    None
172     NaN
173    None
174    None
175    None
176 

In [35]:
space['reused'].isnull().sum()

112

In [36]:
space['reused'].unique()

array([False, None, nan, True], dtype=object)

In [37]:
space["reused"] = space["reused"].fillna("unk")

In [38]:
space['reused'].unique()

array([False, 'unk', True], dtype=object)

In [45]:
space.head(2)

,fairings,links,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,upcoming,cores,auto_update,tbd,launch_library_id,id,reused,recovery_attempt,recovered
0,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False
1,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight'...",True,False,None,5eb87cdaffd86e000604b32b,False,False,False


In [ ]:
#space['date_precision'].unique()
#hemos decidido borrarla porque sus valores no sirven para nada

array(['hour', 'day', 'month'], dtype=object)

In [44]:
space.drop(columns=['date_precision'], inplace= True)

In [40]:
space['date_local'].unique()

array(['2006-03-25T10:30:00+12:00', '2007-03-21T13:10:00+12:00',
       '2008-08-03T15:34:00+12:00', '2008-09-28T11:15:00+12:00',
       '2009-07-13T15:35:00+12:00', '2010-06-04T14:45:00-04:00',
       '2010-12-08T11:43:00-04:00', '2012-05-22T03:44:00-04:00',
       '2012-10-08T20:35:00-04:00', '2013-03-01T15:10:00-04:00',
       '2013-09-29T09:00:00-07:00', '2013-12-03T18:41:00-04:00',
       '2014-01-06T14:06:00-04:00', '2014-04-18T15:25:00-04:00',
       '2014-07-14T11:15:00-04:00', '2014-08-05T04:00:00-04:00',
       '2014-09-07T01:00:00-04:00', '2014-09-21T01:52:00-04:00',
       '2015-01-10T05:47:00-04:00', '2015-02-11T19:03:00-04:00',
       '2015-03-02T23:50:00-04:00', '2015-04-14T16:10:00-04:00',
       '2015-04-27T19:03:00-04:00', '2015-06-28T10:21:00-04:00',
       '2015-12-22T21:29:00-04:00', '2016-01-17T08:42:00-07:00',
       '2016-03-04T19:35:00-04:00', '2016-04-08T16:43:00-04:00',
       '2016-05-06T01:21:00-04:00', '2016-05-27T17:39:00-04:00',
       '2016-06-15T10:29:

In [ ]:
space['date_local'].str.contains('2000', '2010')

In [50]:
space[space["date_local"].str.contains("2007")][["date_local"]]

,date_local
1,2007-03-21T13:10:00+12:00


In [54]:
space['date_local'].min()

'2006-03-25T10:30:00+12:00'

In [55]:
space['date_local'].max()

'2022-12-04T16:00:00-08:00'

In [56]:
space['date_local'].isnull().sum()

0

In [60]:
space['date_local'] = pd.to_datetime(space['date_local'], utc=True)


¿Qué cohetes se han lanzado entre el 2000 y el 2010?

In [69]:
space[space['date_local'].dt.year.between(2000, 2010)][['rocket', 'name', 'date_local']]

,rocket,name,date_local
0,5e9d0d95eda69955f709d1eb,FalconSat,2006-03-24 22:30:00+00:00
1,5e9d0d95eda69955f709d1eb,DemoSat,2007-03-21 01:10:00+00:00
2,5e9d0d95eda69955f709d1eb,Trailblazer,2008-08-03 03:34:00+00:00
3,5e9d0d95eda69955f709d1eb,RatSat,2008-09-27 23:15:00+00:00
4,5e9d0d95eda69955f709d1eb,RazakSat,2009-07-13 03:35:00+00:00
5,5e9d0d95eda69973a809d1ec,Falcon 9 Test Flight,2010-06-04 18:45:00+00:00
6,5e9d0d95eda69973a809d1ec,COTS 1,2010-12-08 15:43:00+00:00


In [62]:
space['date_local'].dtype

datetime64[ns, UTC]


Selecciona aquellas filas donde 'name' sea igual a RazakSat.

In [72]:
space[space['name'].str.contains('Falcon')][['name']]

,name
0,FalconSat
5,Falcon 9 Test Flight
54,Falcon Heavy Test Flight


Selecciona aquellas filas donde 'date_precision' sea igual a day.

In [76]:
space[space['date_precision'].str.contains('day')][['date_precision']]

,date_precision
175,day
192,day
193,day
194,day
198,day



Al resultado anterior le añadimos la condición de que sea el lanzamiento haya ocurrido con éxito.

In [94]:
space[(space['date_precision'].str.contains('day')) & (space['success'].str.contains('Exito'))][['success', 'date_precision', 'rocket']]


,success,date_precision,rocket
175,Exito,day,5e9d0d95eda69973a809d1ec


In [ ]:
exito = space.groupby(['success', 'date_precision'])

In [89]:
exito

In [90]:
exito.ngroups

6

In [91]:
space.groupby(['success', 'date_precision'])['rocket'].count().reset_index()

,success,date_precision,rocket
0,Exito,day,1
1,Exito,hour,180
2,Fracaso,hour,5
3,unk,day,4
4,unk,hour,7
5,unk,month,8


In [77]:
space['success'].unique()

array([ 0.,  1., nan])

In [78]:
mapa = {0:"Fracaso", 1:"Exito"}

In [82]:
space["success"].value_counts(dropna=False, normalize=True)*100

success
1.0    88.292683
NaN     9.268293
0.0     2.439024
Name: proportion, dtype: float64

In [83]:
space["success"] = space["success"].map(mapa)

normalize=True: en lugar de contar valores absolutos, devuelve proporciones (frecuencias relativas entre 0 y 1)

In [84]:
space["success"].value_counts(dropna=False, normalize=True)*100

success
Exito      88.292683
NaN         9.268293
Fracaso     2.439024
Name: proportion, dtype: float64

In [85]:
space["success"] = space["success"].fillna("unk")

In [86]:
space['success'].unique()

array(['Fracaso', 'Exito', 'unk'], dtype=object)


¿Cuál es la ventana de duración media, mínima y máxima por año?

In [111]:
space.groupby('año')['window_min'].agg(['mean', 'max', 'min']).reset_index()

,año,mean,max,min
0,2006,0.000000,0.0,0.0
1,2007,0.000000,0.0,0.0
2,2008,0.000000,0.0,0.0
3,2009,0.000000,0.0,0.0
4,2010,0.000000,0.0,0.0
5,2012,0.000000,0.0,0.0
6,2013,0.000000,0.0,0.0
7,2014,20.000000,120.0,0.0
8,2015,0.000000,0.0,0.0
9,2016,61.875000,120.0,0.0


In [106]:
space['date_local'].str.split('-', expand=True).head()

,0,1,2,3
0,2006,03,25T10:30:00+12:00,None
1,2007,03,21T13:10:00+12:00,None
2,2008,08,03T15:34:00+12:00,None
3,2008,09,28T11:15:00+12:00,None
4,2009,07,13T15:35:00+12:00,None


In [108]:
space[['año','mes', 'dia_hora', 'none']] = space['date_local'].str.split('-', expand=True)

In [109]:
space.head(2)

,fairings,links,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,window_min,año,mes,dia_hora,none
0,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,Fracaso,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a,0.0,2006,03,25T10:30:00+12:00,None
1,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,Fracaso,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight'...",True,False,None,5eb87cdaffd86e000604b32b,0.0,2007,03,21T13:10:00+12:00,None


In [100]:
space['window'].mean()

2568.974358974359

In [101]:
space['window'].min()

0.0

In [102]:
space['window'].max()

18300.0

In [95]:
space.head(2)

,fairings,links,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id
0,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,Fracaso,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a
1,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,Fracaso,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight'...",True,False,None,5eb87cdaffd86e000604b32b


In [96]:
space['window'].unique()

array([    0.,  7200.,  5400.,  2700.,    nan,  9000.,  2940.,  3480.,
        2520., 18300.,  8640.,  8460.,    30.,  7620., 14400.,  6180.,
        1680.,  1560.,  1920.,  7020.,   780.,  5280., 17820.,  5940.,
         900., 18000.])

In [97]:
space["window_min"] = space["window"].apply(lambda celda: celda/60)


In [99]:
space['window_min'].unique()

array([  0. , 120. ,  90. ,  45. ,   nan, 150. ,  49. ,  58. ,  42. ,
       305. , 144. , 141. ,   0.5, 127. , 240. , 103. ,  28. ,  26. ,
        32. , 117. ,  13. ,  88. , 297. ,  99. ,  15. , 300. ])

In [98]:
space[["window", "window_min"]].sample(10)

,window,window_min
178,NaN,NaN
68,0.0,0.0
92,0.0,0.0
91,0.0,0.0
109,NaN,NaN
165,NaN,NaN
66,7200.0,120.0
94,0.0,0.0
23,0.0,0.0
102,0.0,0.0


In [ ]:
#densidad = df_wd.groupby('continent')['density'].sum().reset_index()